# 딥러닝으로_호텔스닷컴리뷰_분석하기_ LSTM_Attention

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical 

import pandas as pd
import numpy as np
import koreanize_matplotlib
from konlpy.tag import Okt

2024-09-13 16:52:29.633830: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-13 16:52:31.858298: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2024-09-13 16:52:31.858413: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2024-09-13 16:52:31.863306: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libd3d12.so
2024-09-13 16:52:32.382741: I tensorflow/c/logging.cc:34] DirectML device enumeration: found 1 compatible adapters.


In [2]:
train_data = pd.read_csv("https://raw.githubusercontent.com/haram4th/data4mdai/main/hotelscom_review_train.csv")
train_data


,description,isgood
0,사람이 너무 많고 작은 수영장과 조식 수용한계로 모두 포기하고 옆의 아이파크몰에서 ...,0
1,방도크고 깨끗하여 아주 좋았어요,1
2,매년여름휴가철마다 찾는곳이예요 너무 좋아요 점점 더 좋아지는듯 직원분들도 너무 친절...,1
3,여수에서는 제일 유명한 호텔이래요 호텔 stay 가 필요하다면 소노캄 여수도 괜찮은...,1
4,가격대비 훌륭하지만 아무래도 오래된 느낌이 많이 드네요겉이불은 세탁하니까 깨끗히나 ...,1
...,...,...
53959,깨끘하고 위치도 좋고 매우 만족스러윘어요다음에 또 이용할 생각입니,1
53960,사촌동생이 고입 시험 보러 올라와서 급히 그 날 하루 묵을 곳이 필요했는데 고급 호...,1
53961,초딩 중딩 하나씩에 4인가족 숙박을했는데 모든것이 다 좋았습니다 철길 옆이긴한데 소...,1
53962,깨끗하고 복층다락방을 아이가 좋아했어요 천장이 높아 방크기에 비해 훨씬 개방감있고 ...,1


In [3]:
test_data = pd.read_csv("./data/hotelscom_review_test.csv")
test_data

,description,isgood
0,쾌적한 시설과 특히 금진온천은 저에게는 특별한추억이었습니다,1
1,관광호텔로 알고 예약 후 직접 방문시 주변이 모두 모텔주변에 위치하였으며 관광호텔급...,0
2,신축이라 그런지 아주 청결합니다 조명은 여러가지 색깔이 들어오는데 다 켜면 잘 조화...,1
3,맨몸으로들어간 호텔에 치솔과 면도기가 없어서 황당했습니다 해외에도 다있고 국내 모텔...,1
4,도어락이 제대로 작동되지 않았고텔레비전도 아주 낡아서 중간에 소리도 안 들리고 화면...,0
...,...,...
35972,신경에 거슬리는 것 없이 편안한 숙소였습니다,1
35973,좋긴한데 들어왔을 때 담배 냄새 비슷한게 나서 조금 그랬어요,1
35974,지금까지 여행가면서 묵었던 숙소 중에 제일 좋았던 곳이예요항상 이런식으로 관리 해주...,1
35975,방에 온도조절이 인돼 너무건조해서 아침에 일어나니 가족들이 모두 코가막힌거외에는 ...,1


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53964 entries, 0 to 53963
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  53964 non-null  object
 1   isgood       53964 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 843.3+ KB


In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35977 entries, 0 to 35976
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  35977 non-null  object
 1   isgood       35977 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 562.3+ KB


## 토큰화하기

In [6]:
docs = train_data['description']
docs

0        사람이 너무 많고 작은 수영장과 조식 수용한계로 모두 포기하고 옆의 아이파크몰에서 ...
1                                        방도크고 깨끗하여 아주 좋았어요
2        매년여름휴가철마다 찾는곳이예요 너무 좋아요 점점 더 좋아지는듯 직원분들도 너무 친절...
3        여수에서는 제일 유명한 호텔이래요 호텔 stay 가 필요하다면 소노캄 여수도 괜찮은...
4        가격대비 훌륭하지만 아무래도 오래된 느낌이 많이 드네요겉이불은 세탁하니까 깨끗히나 ...
                               ...                        
53959                깨끘하고 위치도 좋고 매우 만족스러윘어요다음에 또 이용할  생각입니
53960    사촌동생이 고입 시험 보러 올라와서 급히 그 날 하루 묵을 곳이 필요했는데 고급 호...
53961    초딩 중딩 하나씩에 4인가족 숙박을했는데 모든것이 다 좋았습니다 철길 옆이긴한데 소...
53962    깨끗하고 복층다락방을 아이가 좋아했어요 천장이 높아 방크기에 비해 훨씬 개방감있고 ...
53963    위치 객리단길 가까워 좋음 한옥마을도 걸어서 10분 이내 영화의 거리 중심에 위치침...
Name: description, Length: 53964, dtype: object

In [7]:
okt = Okt()
okt.morphs(docs[0])

['사람',
 '이',
 '너무',
 '많고',
 '작은',
 '수영장',
 '과',
 '조식',
 '수',
 '용한',
 '계',
 '로',
 '모두',
 '포기',
 '하고',
 '옆',
 '의',
 '아이파크몰',
 '에서',
 '그냥',
 '식사',
 '함']

In [8]:
type(docs)

pandas.core.series.Series

In [9]:
# 전체 문장을 토큰화 후 tokenized_docs에 저장 
tokenized_docs = docs.apply(okt.morphs)

In [10]:
# 단어 인덱스 생성 
token = Tokenizer(lower=False)
token.fit_on_texts(tokenized_docs)
print(len(token.word_index))

63571


In [11]:
# 문장 벡터화

X = token.texts_to_sequences(tokenized_docs)
print(X[0])

[147, 1, 10, 362, 381, 124, 24, 22, 39, 8795, 2621, 20, 126, 2252, 36, 179, 13, 3939, 12, 145, 258, 190]


In [12]:
# 가장 긴 문장의 길이 구하기 
max_len = max(len(i) for i in X)
print("가장 긴 문장의 길이(패딩에 사용): ", max_len)

가장 긴 문장의 길이(패딩에 사용):  554


In [13]:
# 패딩
# 패딩을 하면 + 1 해야하지만, 임베딩 때는 + 1  안해도 됨

X_padded = pad_sequences(X, maxlen=max_len, padding='post') 
print(X_padded[0])

# padding='post' 빼고도 출력해보기 

[ 147    1   10  362  381  124   24   22   39 8795 2621   20  126 2252
   36  179   13 3939   12  145  258  190    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [14]:
y= train_data['isgood']
y

0        0
1        1
2        1
3        1
4        1
        ..
53959    1
53960    1
53961    1
53962    1
53963    1
Name: isgood, Length: 53964, dtype: int64

In [15]:
# 홀드아웃 : 데이터 분할

from sklearn.model_selection import train_test_split


In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X_padded, y, test_size=0.3, stratify=y, random_state=10)

In [17]:
# 임베딩에 입력될 단어 수 
word_size = len(token.word_index) + 1
print(word_size)

63572


## 양방향 : RNN 네트워크를 이용해 텍스트 분석

In [18]:
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [19]:
birnn = Sequential()
birnn.add(Embedding(word_size, 64, input_length=max_len))
birnn.add(Bidirectional(SimpleRNN(128, return_sequences = True, activation='tanh')))
birnn.add(Dropout(0.5))
birnn.add(SimpleRNN(64, activation='tanh'))
birnn.add(Dropout(0.5))
birnn.add(Dense(32, activation = 'relu'))
birnn.add(Dense(1, activation='sigmoid'))
birnn.summary()


# activation='tanh' : 탄젠트 

2024-09-13 17:02:40.665537: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-13 17:02:40.668139: I tensorflow/c/logging.cc:34] DirectML: creating device on adapter 0 (AMD Radeon(TM) Graphics)
2024-09-13 17:02:40.774386: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-13 17:02:40.774436: W tensorflow/core/common_runtime/pluggable_device/pluggable_device_bfc_allocator.cc:28] Overriding allow_growth setting because force_memory_growth was requested by the device.
2024-09-13 17:02:40.774461: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 554, 64)           4068608   
                                                                 
 bidirectional (Bidirectiona  (None, 554, 256)         49408     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 554, 256)          0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                20544     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                        

In [20]:
birnn.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
modelpath = "./model/hotels_review_birnn.keras"
checkpoint = ModelCheckpoint(filepath=modelpath, save_best_only=True)
earlystop = EarlyStopping(patience=10)

In [21]:
# birnn_history = birnn.fit(X_train, y_train, epochs=1000, batch_size=280, validation_data=(X_valid, y_valid), callbacks=[earlystop, checkpoint])

IndentationError: unexpected indent (949062613.py, line 2)